In [1]:
!pip install --upgrade pip
!pip install --upgrade pydantic
!pip install --upgrade sqlalchemy
!pip install --force-reinstall --upgrade farm-haystack sentence-transformers pandas
!pip install numpy==1.24.4 --force-reinstall

  Using cached pydantic-2.10.3-py3-none-any.whl.metadata (172 kB)
Using cached pydantic-2.10.3-py3-none-any.whl (456 kB)
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.10.19
    Uninstalling pydantic-1.10.19:
      Successfully uninstalled pydantic-1.10.19
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
farm-haystack 1.26.4 requires pydantic<2, but you have pydantic 2.10.3 which is incompatible.
  Using cached farm_haystack-1.26.4-py3-none-any.whl.metadata (31 kB)
  Using cached sentence_transformers-3.3.1-py3-none-any.whl.metadata (10 kB)
  Using cached pandas-2.2.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (89 kB)
  Using cached boilerpy3-1.0.7-py3-none-any.whl.metadata (5.8 kB)
  Using cached Events-0.5-py3-none-any.whl.metadata (3.9 kB)
  Using cached httpx-0.28.0-py3-none-any.whl.metadata (7.1 kB)
  U

  Using cached numpy-1.24.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.6 kB)
Using cached numpy-1.24.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.1.3
    Uninstalling numpy-2.1.3:
      Successfully uninstalled numpy-2.1.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 1.4.20 requires pydantic>=2.7.0, but you have pydantic 1.10.19 which is incompatible.
cudf-cu12 24.10.1 requires pandas<2.2.3dev0,>=2.0, but you have pandas 2.2.3 which is incompatible.
gensim 4.3.3 requires scipy<1.14.0,>=1.7.0, but you have scipy 1.14.1 which is incompatible.
langchain 0.3.7 requires pydantic<3.0.0,>=2.7.4, but you have pydantic 1.10.19 which is incompatible.


Create a Vector Database of latest news info for fact verification



In [2]:
# any required installations
#!pip install farm-haystack[faiss]
!pip install faiss-gpu
!pip install SQLAlchemy==1.4.48

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 51.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 64.2 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.4.48 which is incompatible.
langchain 0.3.7 requires pydantic<3.0.0,>=2.7.4, but you have pydantic 1.10.19 which is incompatible.


In [3]:
# verify faiss is installed -- needed for vector database
import faiss
print(faiss.__version__)

1.7.2


In [4]:
# required imports
import requests
from sentence_transformers import SentenceTransformer
from haystack.document_stores.faiss import FAISSDocumentStore
from haystack.nodes import EmbeddingRetriever
from haystack.pipelines import DocumentSearchPipeline


In [38]:
# Step 1: Collect trusted news articles from News API
news_api_key = 'ca4a7b633fa443dbab1f09365650e975'
url = f'https://newsapi.org/v2/everything?q=us+technology&sources=reuters,bbc-news,associated-press&from=2024-12-02&to=2024-12-03&apiKey={"ca4a7b633fa443dbab1f09365650e975"}'
response = requests.get(url)
articles = response.json()['articles']

In [39]:
# Step 2: Prepare documents
documents = []
for article in articles:
    content = article['content'] or ''
    documents.append({
        'content': content,
        'meta': {'title': article['title'], 'published_at': article['publishedAt']}
    })

In [40]:
# Step 3: Create a FAISS document store
document_store = FAISSDocumentStore(faiss_index_factory_str="Flat", embedding_dim=384, validate_index_sync=False)

In [41]:
# Step 4: Use Sentence-Transformers to create embeddings
retriever = EmbeddingRetriever(document_store=document_store, embedding_model="sentence-transformers/all-MiniLM-L6-v2")
document_store.write_documents(documents)
document_store.update_embeddings(retriever)

Writing Documents: 10000it [00:00, 437499.11it/s]       


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Updating Embedding:   0%|          | 0/361 [00:00<?, ? docs/s]

Batches:   0%|          | 0/12 [00:00<?, ?it/s]

Documents Processed: 10000 docs [00:12, 821.64 docs/s]


In [42]:
# Step 5: Set up the DocumentSearchPipeline
pipeline = DocumentSearchPipeline(retriever=retriever)

In [44]:
# Step 6: Run a query
query = "Give me AI news"
results = pipeline.run(query=query)

# Print the results in a user-friendly format
for doc in results['documents']:
    print(f"Document ID: {doc.id}, Content: {doc.content[:200]}")  # Print the first 200 characters of each document

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Document ID: c9a9e1843d418502e72662afcd7f71bb, Content: Skip to comments.
Governments Quest to Control AI and Mainatin Its PropagandaArmstrong Economics ^
 | 2 Dec 24
 | Martin Armstrong 
Posted on 12/02/2024 6:18:28 AM PST by delta7
Dear Martin, 
I…
Document ID: 921b6b63a32f681467088aaa254b0467, Content: Sam Sammane is the CEO of TheoSym.
getty
Theres something off about how were adopting AI right now. Everyone wants faster results, but I cant help noticing how often were using these tools without …
Document ID: da7f2adc9d94d1aafcc1b07a2687f15a, Content: Global energy demand is surging, and at the heart of this revolution is the meteoric rise of AI technology. Fueled by the insatiable electricity needs of sprawling data centers, AI is reshaping the e…
Document ID: 68adad513abdebe440c9668cfa0c76a7, Content: Hermen Hulst (left) and Hideaki Nishino (right) have been joint CEOs of Sony since June
PlayStation's boss says artificial intelligence (AI) has the potential to "revolutionise" gam